In [1]:
import cv2 # Computer vision library
import svgutils.transform as sg # svg utility library
import sys # System parameter and functions library
import numpy as np
from tkinter import *
import tkinter as tk

root = tk.Tk()
root.geometry("400x100")
root.title("Selection")
Display = Button(root, height = 7,
                 width = 50,
                 text ="Please select an image/file, click to close.",
                 command = root.destroy)
Display.pack()
                 
mainloop()

from tkinter.filedialog import askopenfilename # tkinter GUI used to select file

path = askopenfilename() # show an "Open" dialog box and return the path to the selected file

# Read the color image
image = cv2.imread(path)

if image is None:
#    sys.exit("Could not read the image or file.")
    cv2.namedWindow("Error, Re-run program", cv2.WINDOW_NORMAL)
    cv2.waitKey(0) # Wait for keypress to continue
    cv2.destroyAllWindows() # Close windows

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display the grayscale image
cv2.imshow('Gray image', gray)  
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows

#----------------------------------------
blurred = cv2.GaussianBlur(gray, (3, 3), 0) # gaussian blur
median = cv2.medianBlur(blurred, 1) # median blur
bilateral = cv2.bilateralFilter(median,3,50,50) # bilateral filter
#----------------------------------------
 
ret, binary = cv2.threshold(bilateral, 255, 255, # threshold of 150
  cv2.THRESH_OTSU) 

# Display the binary image
cv2.imshow('Binary image', binary)
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows

inverted_binary = ~binary
cv2.imshow('Inverted binary image', inverted_binary)
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows

size = np.size(inverted_binary)
skel = np.zeros(inverted_binary.shape,np.uint8)

ret,inverted_binary = cv2.threshold(inverted_binary,127,255,0)
element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
done = False

image = inverted_binary
original = image

while( not done):
    eroded = cv2.erode(image,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(image,temp)
    skel = cv2.bitwise_or(skel,temp)
    image = eroded.copy()

    zeros = size - cv2.countNonZero(image)
    if zeros==size:
        done = True
        
cv2.imshow("original", original)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("skeleton",skel)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("dilate-skeleton",cv2.dilate(skel, element))
cv2.waitKey(0)
cv2.destroyAllWindows()

#kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (35,35))
#morph = cv2.morphologyEx(inverted_binary, cv2.MORPH_CLOSE, kernel, )

contours, hierarchy = cv2.findContours(skel,   #inverted_binary
  cv2.RETR_TREE,
  cv2.CHAIN_APPROX_SIMPLE)


for c in contours:    #revert all back to image
    x,y,w,h = cv2.boundingRect(c)
    cv2.putText(skel, str(w), (x,y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    cv2.rectangle(skel, (x, y), (x + w, y + h), (36,255,12), 1)

cv2.imshow('Result with bounding rectangles (green) and detected contour lines (pink)', skel)
cv2.waitKey(0)
cv2.destroyAllWindows()

with open("path_test.svg", "w+") as f:
    f.write(f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg">')

    for c in contours:
        f.write('<path d="M')
        for i in range(len(c)):
            x, y = c[i][0]
            f.write(f"{x} {y} ")
        f.write('" style="stroke:pink"/>')
    f.write("</svg>")
    
fig = sg.fromfile('path_test.svg')
fig.set_size(('3508','2480')) #'2480','3508' 
fig.save('scaled_test.svg')

with_contours = cv2.drawContours(skel, contours, -1,(255,0,255),3)
cv2.imshow('Detected contours', skel) #with_contours
cv2.waitKey(0)
cv2.destroyAllWindows()

# Show the total number of contours that were detected
root = Tk()
root.geometry("400x100")
root.title("Points")
var = StringVar()
label = Message( root, textvariable=var, relief=RAISED )

var.set("Total number of points: " + str(len(c)))
label.pack()
root.mainloop()


root = Tk()
root.geometry("400x100")
root.title("Contours")
var = StringVar()
label = Message( root, textvariable=var, relief=RAISED )

var.set("Total number of contours detected: " + str(len(contours)))
label.pack()
root.mainloop()

print('Total number of contours detected: ' + str(len(contours)))


Total number of contours detected: 2
